In [1]:
import numpy as np
import pandas as pd

import h2o
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
import plotly.express as px

Preparing data

In [2]:
data = pd.read_csv('data_to_forecast.csv')

In [3]:
## formating date
data['DATE'] = pd.to_datetime(
                          data['DATE'],
                          format='%Y-%m-%d')

In [4]:
## sorting
data = data.sort_values(by="DATE")

In [5]:
list_of_dates = data["DATE"].unique()

In [6]:
list_of_dates

array(['2004-02-11T00:00:00.000000000', '2004-02-18T00:00:00.000000000',
       '2004-02-25T00:00:00.000000000', '2004-03-03T00:00:00.000000000',
       '2004-03-10T00:00:00.000000000', '2004-03-17T00:00:00.000000000',
       '2004-03-24T00:00:00.000000000', '2004-03-31T00:00:00.000000000',
       '2004-04-07T00:00:00.000000000', '2004-04-14T00:00:00.000000000',
       '2004-04-21T00:00:00.000000000', '2004-04-28T00:00:00.000000000',
       '2004-05-05T00:00:00.000000000', '2004-05-12T00:00:00.000000000',
       '2004-05-19T00:00:00.000000000', '2004-05-26T00:00:00.000000000',
       '2004-06-02T00:00:00.000000000', '2004-06-09T00:00:00.000000000',
       '2004-06-16T00:00:00.000000000', '2004-06-23T00:00:00.000000000',
       '2004-06-30T00:00:00.000000000', '2004-07-07T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [7]:
train = data[(data['DATE'] < '2004-06-02')]
test = data[(data['DATE'] >= '2004-06-02')]

In [8]:
cols_to_keep = ['SCORE',"1MReturn"]
train_1m = train.loc[:, cols_to_keep]

In [9]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,59 mins 07 secs
H2O_cluster_timezone:,Europe/Warsaw
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,5 days
H2O_cluster_name:,H2O_from_python_Krzysztof_9n4x71
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.846 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Corelations

In [10]:
cols_to_keep_for_corelation = ["SCORE","SECTOR", "1MReturn", "3MReturn", "6MReturn", "12MReturn"]
#train_1m = train.loc[:, cols_to_keep]
data_to_corelation = data.loc[:, cols_to_keep_for_corelation]

In [11]:
fig = px.imshow(data_to_corelation.corr())
fig.show()

for 1 month returns

In [12]:
train_1m = h2o.H2OFrame(train_1m)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [13]:
x = train_1m.columns
y = "1MReturn"
x.remove(y)

In [14]:

aml = H2OAutoML(max_models=7, seed=1)
aml.train(x=x, y=y, training_frame=train_1m)

AutoML progress: |██
17:53:21.640: AutoML: XGBoost is not available; skipping it.

█████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_1_AutoML_3_20220531_175321

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.007381671351851378
RMSE: 0.08591665351869437
MAE: 0.06230858904765597
RMSLE: 0.09353575968665465
R^2: 0.0933387811416514
Mean Residual Deviance: 0.007381671351851378
Null degrees of freedom: 5863
Residual degrees of freedom: 5861
Null deviance: 47.742331873156864
Residual deviance: 43.28612080725648
AIC: -12135.629349396604

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.008111673158494108
RMSE: 0.09006482753269507
MAE: 0.0651676615365975
RMSLE: 0.09747700617743545
R^2: 0.0036755739584233282
Mean Residual Deviance: 0.008111673158494108
Null degre

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.065284,0.001737,0.062427,0.065503,0.065391,0.065986,0.067113
1,mean_residual_deviance,0.008137,0.000659,0.007046,0.008014,0.008381,0.008665,0.008580
2,mse,0.008137,0.000659,0.007046,0.008014,0.008381,0.008665,0.008580
3,null_deviance,9.550037,0.631418,8.595958,9.704977,9.705296,10.335165,9.408793
4,r2,0.001962,0.002745,0.001498,-0.001241,0.001787,0.006357,0.001409
5,residual_deviance,9.526366,0.618330,8.575536,9.704977,9.687949,10.268034,9.395336
6,rmse,0.090145,0.003729,0.083943,0.089521,0.091546,0.093086,0.092629
7,rmsle,0.096613,0.016091,0.085098,0.088313,0.124719,0.094697,0.090241


In [15]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_BestOfFamily_1_AutoML_3_20220531_175321,0.0900648,0.00811167,0.0651677,0.097477,0.00811167
StackedEnsemble_AllModels_1_AutoML_3_20220531_175321,0.0900711,0.0081128,0.0651794,0.0974778,0.0081128
GLM_1_AutoML_3_20220531_175321,0.0901794,0.00813232,0.0652372,0.0976014,0.00813232
GBM_1_AutoML_3_20220531_175321,0.0902128,0.00813835,0.0653927,0.0976343,0.00813835
GBM_2_AutoML_3_20220531_175321,0.0904252,0.00817672,0.0656122,0.0978358,0.00817672
GBM_3_AutoML_3_20220531_175321,0.090594,0.00820727,0.0657742,0.0979736,0.00820727
GBM_4_AutoML_3_20220531_175321,0.091281,0.00833222,0.0664482,0.0985943,0.00833222
XRT_1_AutoML_3_20220531_175321,0.101982,0.0104004,0.0742613,0.109302,0.0104004
DRF_1_AutoML_3_20220531_175321,0.104001,0.0108163,0.075797,0.110641,0.0108163


3 months forecasting

In [17]:
cols_to_keep_3m = ['SCORE',"3MReturn"]
train_3m = train.loc[:, cols_to_keep_3m]

In [18]:
train_3m = h2o.H2OFrame(train_3m)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [19]:
x_3m = train_3m.columns
y_3m = "3MReturn"
x_3m.remove(y_3m)

In [20]:
aml_3m = H2OAutoML(max_models=7, seed=1)
aml_3m.train(x=x_3m, y=y_3m, training_frame=train_3m)

AutoML progress: |█
17:59:15.69: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_1_AutoML_4_20220531_175915

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.016325576178007978
RMSE: 0.12777157813069376
MAE: 0.09753969705405346
RMSLE: 0.13184091254269506
R^2: 0.17238885788155667
Mean Residual Deviance: 0.016325576178007978
Null degrees of freedom: 5863
Residual degrees of freedom: 5861
Null deviance: 115.67410567091783
Residual deviance: 95.73317870783879
AIC: -7481.179709768114

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.01930067833756554
RMSE: 0.13892688126336653
MAE: 0.10603122861300968
RMSLE: 0.14269482495042762
R^2: 0.021568594673483488
Mean Residual Deviance: 0.01930067833756554
Null degrees 

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.105994,0.003471,0.103720,0.103573,0.109185,0.110341,0.103150
1,mean_residual_deviance,0.019295,0.001518,0.018741,0.017721,0.020603,0.021187,0.018225
2,mse,0.019295,0.001518,0.018741,0.017721,0.020603,0.021187,0.018225
3,null_deviance,23.153040,2.078057,23.321238,21.938042,24.332294,25.767240,20.406390
4,r2,0.020373,0.002783,0.019737,0.018486,0.018206,0.025097,0.020339
5,residual_deviance,22.629496,2.005821,22.807425,21.460133,23.817486,25.106443,19.955996
6,rmse,0.138822,0.005442,0.136897,0.133120,0.143539,0.145557,0.134999
7,rmsle,0.142536,0.007454,0.143288,0.131803,0.146021,0.151836,0.139730


In [21]:
lb_3m = aml_3m.leaderboard
lb_3m.head(rows=lb_3m.nrows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_BestOfFamily_1_AutoML_4_20220531_175915,0.138927,0.0193007,0.106031,0.142695,0.0193007
StackedEnsemble_AllModels_1_AutoML_4_20220531_175915,0.138932,0.0193021,0.106004,0.142695,0.0193021
GBM_2_AutoML_4_20220531_175915,0.139169,0.019368,0.106384,0.142837,0.019368
GBM_3_AutoML_4_20220531_175915,0.139539,0.0194713,0.106533,0.14322,0.0194713
GBM_1_AutoML_4_20220531_175915,0.13958,0.0194825,0.106547,0.143261,0.0194825
GBM_4_AutoML_4_20220531_175915,0.140065,0.0196181,0.106983,0.143689,0.0196181
GLM_1_AutoML_4_20220531_175915,0.140274,0.0196767,0.106789,0.144049,0.0196767
XRT_1_AutoML_4_20220531_175915,0.155516,0.0241851,0.118911,0.158635,0.0241851
DRF_1_AutoML_4_20220531_175915,0.156751,0.0245707,0.11978,0.159837,0.0245707


In [22]:
aml_3m.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_1_AutoML_4_20220531_175915

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.016325576178007978
RMSE: 0.12777157813069376
MAE: 0.09753969705405346
RMSLE: 0.13184091254269506
R^2: 0.17238885788155667
Mean Residual Deviance: 0.016325576178007978
Null degrees of freedom: 5863
Residual degrees of freedom: 5861
Null deviance: 115.67410567091783
Residual deviance: 95.73317870783879
AIC: -7481.179709768114

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.01930067833756554
RMSE: 0.13892688126336653
MAE: 0.10603122861300968
RMSLE: 0.14269482495042762
R^2: 0.021568594673483488
Mean Residual Deviance: 0.01930067833756554
Null degrees of freedom: 5863
Residual degrees of freedom: 5860
Null deviance: 115.7652038014839
Residual deviance: 113.17917777148433
AIC: -6497.503457578093

Cross-Vali

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.105994,0.003471,0.103720,0.103573,0.109185,0.110341,0.103150
1,mean_residual_deviance,0.019295,0.001518,0.018741,0.017721,0.020603,0.021187,0.018225
2,mse,0.019295,0.001518,0.018741,0.017721,0.020603,0.021187,0.018225
3,null_deviance,23.153040,2.078057,23.321238,21.938042,24.332294,25.767240,20.406390
4,r2,0.020373,0.002783,0.019737,0.018486,0.018206,0.025097,0.020339
5,residual_deviance,22.629496,2.005821,22.807425,21.460133,23.817486,25.106443,19.955996
6,rmse,0.138822,0.005442,0.136897,0.133120,0.143539,0.145557,0.134999
7,rmsle,0.142536,0.007454,0.143288,0.131803,0.146021,0.151836,0.139730


For 6 months

In [23]:
cols_to_keep_6m = ['SCORE',"6MReturn"]
train_6m = train.loc[:, cols_to_keep_6m]
train_6m = h2o.H2OFrame(train_6m)
x_6m = train_6m.columns
y_6m = "6MReturn"
x_6m.remove(y_6m)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [24]:
aml_6m = H2OAutoML(max_models=7, seed=1)
aml_6m.train(x=x_6m, y=y_6m, training_frame=train_6m)

AutoML progress: |█
18:11:31.25: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_1_AutoML_5_20220531_181131

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.041524810941009425
RMSE: 0.2037763748352822
MAE: 0.1524815135076402
RMSLE: 0.19598729167857118
R^2: 0.15862225776471361
Mean Residual Deviance: 0.041524810941009425
Null degrees of freedom: 5863
Residual degrees of freedom: 5860
Null deviance: 289.4080496010846
Residual deviance: 243.50149135807928
AIC: -2004.7948129192973

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.047766505603362513
RMSE: 0.21855549776512717
MAE: 0.1630769163167287
RMSLE: 0.20920672859656106
R^2: 0.03215273644182426
Mean Residual Deviance: 0.047766505603362513
Null degrees o

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.162911,0.005886,0.160960,0.160249,0.167719,0.170043,0.155582
1,mean_residual_deviance,0.047720,0.003158,0.046705,0.045768,0.049807,0.052054,0.044267
2,mse,0.047720,0.003158,0.046705,0.045768,0.049807,0.052054,0.044267
3,null_deviance,57.888770,4.925566,58.269245,57.687750,58.600370,64.369870,50.516620
4,r2,0.032370,0.010979,0.024166,0.038664,0.017223,0.041322,0.040474
5,residual_deviance,55.999413,4.808766,56.840416,55.424522,57.576576,61.683563,48.471985
6,rmse,0.218354,0.007196,0.216114,0.213934,0.223174,0.228153,0.210396
7,rmsle,0.208704,0.013334,0.214531,0.196203,0.210760,0.226921,0.195106


In [25]:
lb_6m = aml_6m.leaderboard
lb_6m.head(rows=lb_6m.nrows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_BestOfFamily_1_AutoML_5_20220531_181131,0.218555,0.0477665,0.163077,0.209207,0.0477665
StackedEnsemble_AllModels_1_AutoML_5_20220531_181131,0.218654,0.0478097,0.163231,0.209313,0.0478097
GBM_2_AutoML_5_20220531_181131,0.218709,0.0478337,0.163212,0.209155,0.0478337
GBM_3_AutoML_5_20220531_181131,0.219047,0.0479814,0.163717,0.209421,0.0479814
GBM_1_AutoML_5_20220531_181131,0.219743,0.0482869,0.164968,0.210219,0.0482869
GBM_4_AutoML_5_20220531_181131,0.220278,0.0485225,0.164492,0.210361,0.0485225
GLM_1_AutoML_5_20220531_181131,0.22138,0.0490093,0.165243,0.211896,0.0490093
XRT_1_AutoML_5_20220531_181131,0.247819,0.0614141,0.184626,0.234376,0.0614141
DRF_1_AutoML_5_20220531_181131,0.252008,0.0635079,0.187416,0.237808,0.0635079


In [26]:
aml_6m.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_1_AutoML_5_20220531_181131

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.041524810941009425
RMSE: 0.2037763748352822
MAE: 0.1524815135076402
RMSLE: 0.19598729167857118
R^2: 0.15862225776471361
Mean Residual Deviance: 0.041524810941009425
Null degrees of freedom: 5863
Residual degrees of freedom: 5860
Null deviance: 289.4080496010846
Residual deviance: 243.50149135807928
AIC: -2004.7948129192973

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.047766505603362513
RMSE: 0.21855549776512717
MAE: 0.1630769163167287
RMSLE: 0.20920672859656106
R^2: 0.03215273644182426
Mean Residual Deviance: 0.047766505603362513
Null degrees of freedom: 5863
Residual degrees of freedom: 5860
Null deviance: 289.44385933996404
Residual deviance: 280.1027888581178
AIC: -1183.6379485268699

Cross-Vali

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.162911,0.005886,0.160960,0.160249,0.167719,0.170043,0.155582
1,mean_residual_deviance,0.047720,0.003158,0.046705,0.045768,0.049807,0.052054,0.044267
2,mse,0.047720,0.003158,0.046705,0.045768,0.049807,0.052054,0.044267
3,null_deviance,57.888770,4.925566,58.269245,57.687750,58.600370,64.369870,50.516620
4,r2,0.032370,0.010979,0.024166,0.038664,0.017223,0.041322,0.040474
5,residual_deviance,55.999413,4.808766,56.840416,55.424522,57.576576,61.683563,48.471985
6,rmse,0.218354,0.007196,0.216114,0.213934,0.223174,0.228153,0.210396
7,rmsle,0.208704,0.013334,0.214531,0.196203,0.210760,0.226921,0.195106


12 months

In [27]:
cols_to_keep_12m = ['SCORE',"12MReturn"]
train_12m = train.loc[:, cols_to_keep_12m]
train_12m = h2o.H2OFrame(train_12m)
x_12m = train_12m.columns
y_12m = "12MReturn"
x_12m.remove(y_12m)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [29]:
aml_12m = H2OAutoML(max_models=7, seed=1)
aml_12m.train(x=x_12m, y=y_12m, training_frame=train_12m)

AutoML progress: |█
18:28:07.364: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_1_AutoML_7_20220531_182807

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.10069957835401587
RMSE: 0.3173319686921188
MAE: 0.22534389611057756
RMSLE: 0.2659935331875712
R^2: 0.17340134911807226
Mean Residual Deviance: 0.10069957835401587
Null degrees of freedom: 5863
Residual degrees of freedom: 5860
Null deviance: 714.3761084510802
Residual deviance: 590.502327467949
AIC: 3189.832577559926

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.11750409250785605
RMSE: 0.3427886995043099
MAE: 0.2425937338208772
RMSLE: 0.2851254644810321
R^2: 0.03546046639204792
Mean Residual Deviance: 0.11750409250785605
Null degrees of freedom

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.242412,0.010355,0.242334,0.241176,0.252491,0.250014,0.226042
1,mean_residual_deviance,0.117291,0.009697,0.114406,0.121166,0.123808,0.125416,0.101662
2,mse,0.117291,0.009697,0.114406,0.121166,0.123808,0.125416,0.101662
3,null_deviance,142.929660,15.402166,144.145220,151.627780,146.954390,155.433960,116.486960
4,r2,0.035155,0.007801,0.034075,0.031127,0.024880,0.041817,0.043877
5,residual_deviance,137.804550,15.234646,139.231960,146.731480,143.121930,148.617740,111.319640
6,rmse,0.342235,0.014421,0.338239,0.348088,0.351863,0.354141,0.318844
7,rmsle,0.284370,0.019683,0.283895,0.271425,0.291291,0.313241,0.262000


In [30]:
lb_12m = aml_12m.leaderboard
lb_12m.head(rows=lb_12m.nrows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_BestOfFamily_1_AutoML_7_20220531_182807,0.342789,0.117504,0.242594,0.285125,0.117504
StackedEnsemble_AllModels_1_AutoML_7_20220531_182807,0.342852,0.117548,0.242665,0.2851,0.117548
GBM_2_AutoML_7_20220531_182807,0.343295,0.117852,0.243133,0.285215,0.117852
GBM_3_AutoML_7_20220531_182807,0.344369,0.11859,0.244175,0.286137,0.11859
GBM_1_AutoML_7_20220531_182807,0.344734,0.118841,0.244131,0.286418,0.118841
GBM_4_AutoML_7_20220531_182807,0.34512,0.119108,0.24476,0.28664,0.119108
GLM_1_AutoML_7_20220531_182807,0.347568,0.120803,0.245149,0.288904,0.120803
XRT_1_AutoML_7_20220531_182807,0.385479,0.148594,0.273389,0.314991,0.148594
DRF_1_AutoML_7_20220531_182807,0.390944,0.152837,0.276827,0.318843,0.152837


In [31]:
aml_12m.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_1_AutoML_7_20220531_182807

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.10069957835401587
RMSE: 0.3173319686921188
MAE: 0.22534389611057756
RMSLE: 0.2659935331875712
R^2: 0.17340134911807226
Mean Residual Deviance: 0.10069957835401587
Null degrees of freedom: 5863
Residual degrees of freedom: 5860
Null deviance: 714.3761084510802
Residual deviance: 590.502327467949
AIC: 3189.832577559926

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.11750409250785605
RMSE: 0.3427886995043099
MAE: 0.2425937338208772
RMSLE: 0.2851254644810321
R^2: 0.03546046639204792
Mean Residual Deviance: 0.11750409250785605
Null degrees of freedom: 5863
Residual degrees of freedom: 5860
Null deviance: 714.6483025787527
Residual deviance: 689.0439984660679
AIC: 4094.8327883186053

Cross-Validation Metri

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.242412,0.010355,0.242334,0.241176,0.252491,0.250014,0.226042
1,mean_residual_deviance,0.117291,0.009697,0.114406,0.121166,0.123808,0.125416,0.101662
2,mse,0.117291,0.009697,0.114406,0.121166,0.123808,0.125416,0.101662
3,null_deviance,142.929660,15.402166,144.145220,151.627780,146.954390,155.433960,116.486960
4,r2,0.035155,0.007801,0.034075,0.031127,0.024880,0.041817,0.043877
5,residual_deviance,137.804550,15.234646,139.231960,146.731480,143.121930,148.617740,111.319640
6,rmse,0.342235,0.014421,0.338239,0.348088,0.351863,0.354141,0.318844
7,rmsle,0.284370,0.019683,0.283895,0.271425,0.291291,0.313241,0.262000
